<a href="https://colab.research.google.com/github/youngsunjang/Class_DSU_OperatingSystem/blob/main/Skip_gram_OpenMP_Sequential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Program

In [1]:
import torch
torch.manual_seed(10)
from torch.autograd import Variable
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from sklearn import decomposition
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10,8)
import nltk
from nltk.corpus import stopwords
import time
torch.set_num_threads(1)  # Adjust this value in paranthesis for changing the number of threads

In [2]:
######################################
# Passage split
######################################
import nltk

# Download the punkt tokenizer if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# Specify the path to your text file
file_path = "/content/New_data.txt"

# Read the passage from the text file
with open(file_path, 'r', encoding='utf-8') as file:
    passage = file.read()

# Use nltk to tokenize the text into sentences
corpus = nltk.sent_tokenize(passage)

In [5]:
def create_vocabulary(corpus):
    '''Creates a dictionary with all unique words in corpus with id'''
    vocabulary = {}
    i = 0
    for s in corpus:
        for w in s.split():
            if w not in vocabulary:
                vocabulary[w] = i
                i+=1
    return vocabulary

def prepare_set(corpus, n_gram = 1):
    '''Creates a dataset with Input column and Outputs columns for neighboring words.
       The number of neighbors = n_gram*2'''
    columns = ['Input'] + [f'Output{i+1}' for i in range(n_gram*2)]
    result = pd.DataFrame(columns = columns)
    for sentence in corpus:
        for i,w in enumerate(sentence.split()):
            inp = [w]
            out = []
            for n in range(1,n_gram+1):
                # look back
                if (i-n)>=0:
                    out.append(sentence.split()[i-n])
                else:
                    out.append('<padding>')

                # look forward
                if (i+n)<len(sentence.split()):
                    out.append(sentence.split()[i+n])
                else:
                    out.append('<padding>')
            row = pd.DataFrame([inp+out], columns = columns)
            result = result.append(row, ignore_index = True)
    return result

In [6]:
def prepare_set_ravel(corpus, n_gram = 1):
    '''Creates a dataset with Input column and Output column for neighboring words.
       The number of neighbors = n_gram*2'''
    columns = ['Input', 'Output']
    result = pd.DataFrame(columns = columns)
    for sentence in corpus:
        for i,w in enumerate(sentence.split()):
            inp = w
            for n in range(1,n_gram+1):
                # look back
                if (i-n)>=0:
                    out = sentence.split()[i-n]
                    row = pd.DataFrame([[inp,out]], columns = columns)
                    result = result.append(row, ignore_index = True)

                # look forward
                if (i+n)<len(sentence.split()):
                    out = sentence.split()[i+n]
                    row = pd.DataFrame([[inp,out]], columns = columns)
                    result = result.append(row, ignore_index = True)
    return result

In [7]:
stop_words = set(stopwords.words('english'))

In [8]:
def preprocess(corpus):
    result = []
    for i in corpus:
        out = nltk.word_tokenize(i)
        out = [x.lower() for x in out]
        out = [x for x in out if x not in stop_words]
        result.append(" ". join(out))
    return result

#########################
# In paper, they used 300 dimensions and 5 context (n gram)
#########################

corpus = preprocess(corpus)
vocabulary = create_vocabulary(corpus)
train_emb = prepare_set(corpus, n_gram = 2)
train_emb = prepare_set_ravel(corpus, n_gram = 2)
train_emb.Input = train_emb.Input.map(vocabulary)
train_emb.Output = train_emb.Output.map(vocabulary)

vocab_size = len(vocabulary)

def get_input_tensor(tensor):
    '''Transform 1D tensor of word indexes to one-hot encoded 2D tensor'''
    size = [*tensor.shape][0]
    inp = torch.zeros(size, vocab_size).scatter_(1, tensor.unsqueeze(1), 1.)
    return Variable(inp).float()

embedding_dims = 300
device = torch.device('cpu')

initrange = 0.5 / embedding_dims
W1 = Variable(torch.randn(vocab_size, embedding_dims, device=device).uniform_(-initrange, initrange).float(), requires_grad=True) # shape V*H
W2 = Variable(torch.randn(embedding_dims, vocab_size, device=device).uniform_(-initrange, initrange).float(), requires_grad=True) #shape H*V
print(f'W1 shape is: {W1.shape}, W2 shape is: {W2.shape}')

num_epochs = 2000
learning_rate = 10.0
lr_decay = 0.99
loss_hist = []

# Record the start time
start_time = time.time()

W1 shape is: torch.Size([478, 300]), W2 shape is: torch.Size([300, 478])


In [9]:
for epo in range(num_epochs):
    total_correct = 0
    total_samples = 0

    for x,y in zip(DataLoader(train_emb.Input.values, batch_size=train_emb.shape[0]), DataLoader(train_emb.Output.values, batch_size=train_emb.shape[0])):
        # one-hot encode input tensor
        input_tensor = get_input_tensor(x).to(device) #shape N*V

        # simple NN architecture
        h = input_tensor.mm(W1.to(device)) # shape 1*H
        y_pred = h.mm(W2.to(device)) # shape 1*V

        # move target tensor to the same device
        y = y.to(device)

        # define loss func
        loss_f = torch.nn.CrossEntropyLoss() # see details: https://pytorch.org/docs/stable/nn.html

        #compute loss
        loss = loss_f(y_pred, y)

        # bakpropagation step
        loss.backward()

        # Update weights using gradient descent. For this step we just want to mutate
        # the values of w1 and w2 in-place; we don't want to build up a computational
        # graph for the update steps, so we use the torch.no_grad() context manager
        # to prevent PyTorch from building a computational graph for the updates
        with torch.no_grad():
            # SGD optimization is implemented in PyTorch, but it's very easy to implement manually providing better understanding of process
            W1 -= learning_rate*W1.grad.data
            W2 -= learning_rate*W2.grad.data
            # zero gradients for next step
            W1.grad.data.zero_()
            W2.grad.data.zero_()

        # compute loss and accuracy
        _, predicted = torch.max(y_pred.data, 1)
        total_correct += (predicted == y).sum().item()
        total_samples += y.size(0)

    if epo%10 == 0:
        learning_rate *= lr_decay
    loss_hist.append(loss)
    if epo%50 == 0:
        accuracy = total_correct / total_samples
        print(f'Epoch {epo}, loss = {loss}, accuracy = {accuracy}')

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Elapsed Time: {elapsed_time} seconds")

Epoch 0, loss = 6.169610023498535, accuracy = 0.0008520306731042318
Epoch 50, loss = 5.466715335845947, accuracy = 0.25901732462368643
Epoch 100, loss = 5.735353946685791, accuracy = 0.2588753195115024
Epoch 150, loss = 4.822391510009766, accuracy = 0.21144561204203352
Epoch 200, loss = 6.614449977874756, accuracy = 0.15805168986083498
Epoch 250, loss = 4.4044575691223145, accuracy = 0.17196819085487078
Epoch 300, loss = 4.49868106842041, accuracy = 0.22337404146549275
Epoch 350, loss = 4.3018412590026855, accuracy = 0.21854586765123543
Epoch 400, loss = 4.342555522918701, accuracy = 0.2188298778756035
Epoch 450, loss = 4.45860481262207, accuracy = 0.1621698381141721
Epoch 500, loss = 4.304757118225098, accuracy = 0.21740982675376314
Epoch 550, loss = 4.532461166381836, accuracy = 0.16259585345072422
Epoch 600, loss = 4.564937591552734, accuracy = 0.16245384833854018
Epoch 650, loss = 4.05201530456543, accuracy = 0.28358420903152515
Epoch 700, loss = 3.998051643371582, accuracy = 0.273